In [115]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [116]:
raw_data = pd.read_csv('./data/FDA_APPROVED.csv')

In [117]:
raw_data.head()

,Generic Name,BDDCS Class,Maximum Strength Dose Value,Maximum Strength Dose Unit,Formulation,Route,Measured Solubility (mg/mL),Dose Number,% Excreted Unchanged in Urine,MW Drug,...,ALOGPS 2.1 solubility,cDose Number (ALOGPS based),minVSLgS 3-7.5,cDose Number (minVSLgS based),CLogP,HBA,HBD,PSA,Ro5,PSDA
0,Abacavir Sulfate,1,300.0,mg,tablets,oral,77.00,0.02,1.2,286.34,...,-2.37,1.000,-3.07,4.100000e+00,0.81,6,3,95.80,0,2.99
1,Acarbose,1,100.0,mg,tablets,oral,NaN,NaN,1.0,645.62,...,-0.64,0.003,3.30,3.000000e-07,-6.66,19,14,351.80,3,1.84
2,Acebutolol Hydrochloride,1,400.0,mg,capsules,oral,NaN,NaN,10.0,336.43,...,-3.27,8.900,-1.46,1.000000e-01,1.71,5,3,97.05,0,3.47
3,Acetaminophen; Paracetamol,1,1000.0,mg,tablets,oral,23.70,0.20,3.0,151.17,...,-1.55,0.900,-1.66,1.200000e+00,0.49,2,2,55.41,0,2.73
4,Acetohexamide,1,500.0,mg,tablets,oral,3.43,0.60,NaN,324.40,...,-3.85,44.000,-3.13,8.200000e+00,2.25,4,2,103.22,0,3.14


In [118]:
pick_data = pd.DataFrame({'Unchanged_excretion_in_urine':raw_data['% Excreted Unchanged in Urine'], 
     'cLogP':raw_data['CLogP'], 'HBA':raw_data['HBA'], 'HBD': raw_data['HBD'], 
                            'PSDA':raw_data['PSDA'], 'Formulation':raw_data['Formulation']})

In [119]:
pick_data.head()

,Unchanged_excretion_in_urine,cLogP,HBA,HBD,PSDA,Formulation
0,1.2,0.81,6,3,2.99,tablets
1,1.0,-6.66,19,14,1.84,tablets
2,10.0,1.71,5,3,3.47,capsules
3,3.0,0.49,2,2,2.73,tablets
4,NaN,2.25,4,2,3.14,tablets


In [120]:
pick_data = picked_data.dropna() # RFC should be able to handle missing value. why I have to do this one?

In [121]:
pick_data.head()

,Unchanged_excretion_in_urine,cLogP,HBA,HBD,PSDA,Formulation
0,1.2,0.81,6,3,2.99,tablets
1,1.0,-6.66,19,14,1.84,tablets
2,10.0,1.71,5,3,3.47,capsules
3,3.0,0.49,2,2,2.73,tablets
5,1.4,1.02,3,1,2.64,tablets


In [127]:
X = pick_data[['Unchanged_excretion_in_urine','cLogP','HBA', 'HBD', 'PSDA']]
Y = pick_data['Formulation']

In [128]:
X.head()

,Unchanged_excretion_in_urine,cLogP,HBA,HBD,PSDA
0,1.2,0.81,6,3,2.99
1,1.0,-6.66,19,14,1.84
2,10.0,1.71,5,3,3.47
3,3.0,0.49,2,2,2.73
5,1.4,1.02,3,1,2.64


In [140]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state = 1)
RFC = RandomForestClassifier(n_estimators=100, random_state=2)
Classicifation = RFC.fit(x_train, y_train)
y_pred = Classicifation.predict(x_test)
importance = pd.Series(Classicifation.feature_importances_,
                       index=['Unchanged_excretion_in_urine','cLogP','HBA', 'HBD', 'PSDA']).sort_values(ascending=False)
print("Accuracy:",accuracy_score(y_test, y_pred))
print("Feature Importance", importance)
print("Classicifation report", classification_report(y_test,y_pred))

Accuracy: 0.7012987012987013
Feature Importance cLogP                           0.263385
PSDA                            0.262081
Unchanged_excretion_in_urine    0.192449
HBA                             0.171676
HBD                             0.110409
dtype: float64
Classicifation report               precision    recall  f1-score   support

    capsules       0.33      0.11      0.17         9
    solution       0.60      0.40      0.48        15
     tablets       0.73      0.89      0.80        53

    accuracy                           0.70        77
   macro avg       0.56      0.47      0.48        77
weighted avg       0.66      0.70      0.67        77

